In [ ]:
!pip install spacy==3.3.0
!pip install spacy-transformers

In [ ]:
import re
import glob
import json
import spacy
import spacy.cli
import pandas as pd
from spacy.tokens import DocBin
from tqdm import tqdm

# spacy.cli.download('en_core_web_md')

In [ ]:
nlp = spacy.load('en_core_web_md')
db = DocBin()

Text Category Training

In [ ]:
df = pd.read_csv('spacy_training/train.csv')
df['text'] = df['text'].astype(str)
df['label'] = df['label'].astype(str)

In [ ]:
train = []
for i in df.index:
    text = df['text'][i]
    train.append((text, df['label'][i]))

train

In [ ]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        if label == 'no':
            doc.cats['experience'] = 0
            doc.cats['no'] = 1
        else:
            doc.cats['experience'] = 1
            doc.cats['no'] = 0
        docs.append(doc)
    return docs

In [ ]:
train_docs = make_docs(train)
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk('./textcat_train.spacy')

Named Entity Recognition Training

In [ ]:
data = []
train = []

for file in glob.glob('path to json'):
  f = open(file)
  data.append(json.load(f))

for i in range(len(data)):
    for j in data[0]['annotations']:
        train.append(tuple(j))

In [ ]:
for text, annot in tqdm(train):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot['entities']:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      print("Skip")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

db.to_disk('./ner_data.spacy')

In [ ]:
!python -m spacy init fill-config ../input/baseconfigcfg/base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./validation_data.spacy --gpu-id 0

In [ ]:
nlp.to_disk()